In [15]:
import numpy as np
import pandas as pd

In [16]:
data = pd.read_csv('Bitcoin Historical Data(2010-2022).csv')
data

,Date,Open,High,Low,Close,Volume
0,2010-07-18,0.0,0.1,0.1,0.1,75
1,2010-07-19,0.1,0.1,0.1,0.1,574
2,2010-07-20,0.1,0.1,0.1,0.1,262
3,2010-07-21,0.1,0.1,0.1,0.1,575
4,2010-07-22,0.1,0.1,0.1,0.1,2160
...,...,...,...,...,...,...
4301,2022-04-27,38113.0,39447.0,37869.0,39243.0,426830240
4302,2022-04-28,39243.0,40382.0,38888.0,39748.0,542391872
4303,2022-04-29,39748.0,39919.0,38175.0,38581.0,382904352
4304,2022-04-30,38590.0,38776.0,37596.0,37650.0,248220336


In [17]:
from datetime import datetime


data.columns = [d.lower() for d in data]

In [18]:
data.isna().sum()

date      0
open      0
high      0
low       0
close     0
volume    0
dtype: int64

In [19]:
data.date = np.array([datetime.strptime(d, '%Y-%m-%d') for d in data.date])

In [20]:
data.sort_values(by='date', inplace=True)

In [21]:
data = data[data.date >= datetime(2021, 2, 5)]
data.reset_index(drop=True, inplace=True)
data

,date,open,high,low,close,volume
0,2021-02-05,36976.4,38303.6,36640.0,38297.6,100604
1,2021-02-06,38300.6,40939.7,38275.3,39256.6,156019
2,2021-02-07,39228.9,39691.7,37381.6,38852.9,125369
3,2021-02-08,38853.6,46596.4,38057.0,46395.7,248473
4,2021-02-09,46396.4,48143.5,45104.7,46508.6,193874
...,...,...,...,...,...,...
446,2022-04-27,38113.0,39447.0,37869.0,39243.0,426830240
447,2022-04-28,39243.0,40382.0,38888.0,39748.0,542391872
448,2022-04-29,39748.0,39919.0,38175.0,38581.0,382904352
449,2022-04-30,38590.0,38776.0,37596.0,37650.0,248220336
